Kelompok :
Eunice Cristine
Fasya Ghassani Hadiyan
Ferry Abu Darrin Alghifari

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
!pwd
os.chdir("/content/drive/My Drive")
os.chdir("/")
!pwd
!ls

/content
/
bin	 datalab  home	 lib64	opt   run   sys		       tools
boot	 dev	  lib	 media	proc  sbin  tensorflow-1.15.2  usr
content  etc	  lib32  mnt	root  srv   tmp		       var


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark-2.4.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark
!jps

408 SparkSubmit
474 Jps


In [ ]:
import pyspark
from pyspark.context import SparkContext
spark = SparkSession\
        .builder\
        .appName("Map Pada Spark")\
        .getOrCreate()
        
#sc = SparkContext.getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [12]:
from google.colab import files
files.upload()

Saving srr.csv to srr.csv


{'srr.csv': b'usia,berat,tinggi,leher,dada,perut,pinggul,paha,lutut,pk,bisep,lb,pt,lemak\r\n27,274,29,45,86,103,89,81,38,31,42,23,16,16\r\n34,269,66,31,84,100,143,67,46,20,39,32,17,24\r\n23,224,76,45,107,117,117,47,34,30,33,29,15,0\r\n39,250,33,36,112,97,140,64,46,23,41,23,15,7\r\n52,276,72,47,95,138,130,61,34,26,43,28,20,32\r\n23,158,62,40,110,134,111,73,41,24,39,24,16,26\r\n69,214,42,49,98,122,86,64,34,23,37,32,15,2\r\n65,266,46,39,124,96,116,52,45,24,40,28,15,13\r\n25,324,46,44,135,100,86,47,37,29,33,31,17,6\r\n80,324,76,37,112,96,124,61,33,23,33,31,18,36\r\n43,283,35,48,119,94,134,67,44,19,44,30,15,26\r\n68,297,68,50,82,109,119,64,44,24,40,24,16,46\r\n56,254,51,39,110,127,101,72,34,25,42,30,19,31\r\n49,177,71,33,112,111,126,52,40,29,28,26,16,0\r\n22,259,34,34,93,115,144,62,40,29,34,21,18,15\r\n30,184,71,46,108,140,101,67,33,29,35,26,18,34\r\n53,256,44,38,88,91,140,83,39,23,42,24,17,7\r\n76,248,54,34,127,140,133,85,42,20,39,32,17,28\r\n41,228,44,34,102,97,133,77,34,27,30,25,16,3\r\n

time: 10.4 s (started: 2021-08-15 11:41:41 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 183 µs (started: 2021-08-15 09:37:16 +00:00)


In [11]:
from pyspark import SQLContext, SparkConf, SparkContext
from pyspark.sql import SparkSession
import numpy as np # linear algebra
import matplotlib.pyplot as plt

sc = SparkContext.getOrCreate()
if (sc is None):
    sc = SparkContext(master="local[*]", appName="Linear Regression")
spark = SparkSession(sparkContext=sc)
sqlcontext = SQLContext(sc)
data = sqlcontext.read.csv('./s.csv', header = True, inferSchema = True)
from pyspark.ml.feature import VectorAssembler
# mendifinisikan Salary sebagai variabel label/predictor
dataset= data.select(data.usia , data.berat,  data.tinggi, data.leher, data.dada, data.perut, data.pinggul, data.paha, data.lutut, data.pk, data.bisep, data.lb, data.pt, data.lemak.alias('label'))

# split data menjadi 70% training and 30% testing 
training, test = dataset.randomSplit([0.75, 0.25], seed = 100)
# mengubah fitur menjadi vektor
assembler = VectorAssembler(inputCols =['usia','berat','tinggi','dada', 'perut', 'pinggul', 'paha', 'lutut','pk','bisep','lb','pt'],
                            outputCol='features')
trainingSet = assembler.transform(training)
# memilih kolom yang akan di vektorisasi
trainingSet = trainingSet.select("features","label")
from pyspark.ml.regression import LinearRegression
# fit data training ke model
lr = LinearRegression()
lr_Model = lr.fit(trainingSet)
# assembler : fitur menjadi vektor
testSet = assembler.transform(test)
# memilih kolom fitur dan label
testSet = testSet.select("features", "label")
# fit testing data ke model linear regression
testSet = lr_Model.transform(testSet)
testSet.show(truncate=False)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
y=evaluator.evaluate(testSet, {evaluator.metricName: "r2"})
print(y)


+------------------------------------------------------------------+-----+------------------+
|features                                                          |label|prediction        |
+------------------------------------------------------------------+-----+------------------+
|[23.0,188.15,77.5,96.6,85.3,102.5,59.1,37.6,23.2,31.8,29.7,18.3]  |10.3 |10.148028894295244|
|[24.0,184.25,71.25,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7]|28.7 |25.108303120874417|
|[24.0,208.5,72.75,102.0,99.1,110.1,71.2,43.5,25.2,36.1,30.3,18.7] |19.2 |23.16805743340708 |
|[24.0,210.25,74.75,104.5,94.4,107.8,66.0,42.0,25.6,35.7,30.6,18.8]|20.9 |17.21429220206459 |
|[26.0,181.0,69.75,105.1,90.7,100.3,58.4,38.3,22.9,31.9,27.8,17.7] |19.2 |16.5214116581756  |
|[26.0,223.0,70.25,114.1,106.8,113.9,67.6,42.7,24.7,36.0,30.4,18.4]|25.2 |27.479499541478596|
|[26.0,241.75,74.5,108.3,102.9,114.4,72.9,43.5,25.1,38.5,33.8,19.6]|19.6 |22.854422147151304|
|[27.0,146.0,72.25,88.5,72.8,91.1,53.6,36.8,23.8,27.8,26.3,1

In [13]:
from pyspark import SQLContext, SparkConf, SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
if (sc is None):
    sc = SparkContext(master="local[*]", appName="Linear Regression")
spark = SparkSession(sparkContext=sc)
sqlcontext = SQLContext(sc)
data = sqlcontext.read.csv('./srr.csv', header = True, inferSchema = True)
from pyspark.ml.feature import VectorAssembler
# mendifinisikan Salary sebagai variabel label/predictor
dataset= data.select(data.usia , data.berat,  data.tinggi, data.leher, data.dada, data.perut, data.pinggul, data.paha, data.lutut, data.pk, data.bisep, data.lb, data.pt, data.lemak.alias('label'))

# split data menjadi 70% training and 30% testing 
training, test = dataset.randomSplit([0.75, 0.25], seed = 100)
# mengubah fitur menjadi vektor
assembler = VectorAssembler(inputCols =['usia','berat','tinggi','dada', 'perut', 'pinggul', 'paha', 'lutut','pk','bisep','lb','pt'],
                            outputCol='features')
trainingSet = assembler.transform(training)
# memilih kolom yang akan di vektorisasi
trainingSet = trainingSet.select("features","label")
from pyspark.ml.regression import LinearRegression
# fit data training ke model
lr = LinearRegression()
lr_Model = lr.fit(trainingSet)
# assembler : fitur menjadi vektor
testSet = assembler.transform(test)
# memilih kolom fitur dan label
testSet = testSet.select("features", "label")
# fit testing data ke model linear regression
testSet = lr_Model.transform(testSet)
testSet.show(truncate=False)

from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
print(evaluator.evaluate(testSet, {evaluator.metricName: "r2"}))

+-----------------------------------------------------------------+-----+------------------+
|features                                                         |label|prediction        |
+-----------------------------------------------------------------+-----+------------------+
|[22.0,264.0,45.0,134.0,139.0,88.0,76.0,44.0,31.0,35.0,24.0,16.0] |27   |21.026042672077928|
|[23.0,158.0,62.0,110.0,134.0,111.0,73.0,41.0,24.0,39.0,24.0,16.0]|26   |19.39967784088317 |
|[23.0,194.0,69.0,100.0,143.0,107.0,54.0,47.0,30.0,43.0,33.0,18.0]|43   |20.241266060106966|
|[23.0,224.0,76.0,107.0,117.0,117.0,47.0,34.0,30.0,33.0,29.0,15.0]|0    |22.17052338974672 |
|[24.0,345.0,48.0,96.0,115.0,110.0,77.0,36.0,23.0,25.0,28.0,18.0] |38   |24.068351504007826|
|[27.0,178.0,52.0,106.0,73.0,89.0,57.0,48.0,27.0,40.0,29.0,19.0]  |27   |19.03490063646843 |
|[27.0,205.0,71.0,104.0,106.0,90.0,48.0,34.0,29.0,44.0,33.0,16.0] |15   |23.49096988813782 |
|[27.0,241.0,52.0,117.0,80.0,139.0,64.0,48.0,25.0,36.0,28.0,16.0] |7  

-> running waktu untuk membandingkan dengan pandas/sklearn